# Support Vector Machine

The one problem with the kernelized SVM is that it's training time scales as $O(n_{\text{sample}}^\alpha)$, with scaling exponent between 2 and 3.

I've come across a couple approximate methods.  The first is a bagged model using an ensemble of SVM's based on subsets of the data. That leverages the existing (presumably smarter) scikit-learn code, in a way that could scale up.  Each classifier is then part of an ensemble, and we determine the class by majority vote. 
This in essence assumes that the Kernel matrix is block-diagonal, since this ensemble misses any correlations between the different subsets.
One way would be to take multiple random batches of data.

Or use an approximate kernel via Random Fourier Components.  The resulting SVM uses the linear library (but requires $N_{\text{sample}}$ samples per Kernel estimate.)  This method can be applied to any method where the kernel function $K(x,y)$ has a known Fourier transform (which can be
efficiently sampled from).

In [ ]:
df_com=pd.read_csv('saved_dataframes/cleaned_comments.tsv.gzip',sep='\t',compression='gzip')
train_msk=df_com['split']=='train'
df_train=df_com[train_msk]
df_dev=df_com[df_com['split']=='dev']
df_test=df_com[df_com['split']=='test']

count_vect=CountVectorizer(stop_words='english',lowercase=True,strip_accents='unicode')
tfidf_vect=TfidfVectorizer(stop_words='english',lowercase=True,strip_accents='unicode')
X_train_counts=count_vect.fit_transform(df_train['comment_clean'])
X_train_tfidf=tfidf_vect.fit_transform(df_train['comment_clean'])

#do the same transformations using existing vocab built up in training.
X_dev_tfidf=tfidf_vect.transform(df_dev['comment_clean'])
X_test_tfidf=tfidf_vect.transform(df_test['comment_clean'])

X_dev_counts=count_vect.transform(df_dev['comment_clean'])
X_test_counts=count_vect.transform(df_test['comment_clean'])


# SVM Ensemble

Since apparently the training time for a SVM goes as $O(n_{sample}^{3})$, maybe it is better to train an ensemble of SVMs.
In which case the training time is $O(n_{sample}^{3}/n_{ensemble^{2}})$ for the ensemble.  Then evaluating the results typically takes $O(n_sample)$ for all of the ensemble together.  (This is something like making the crude assumption that the kernels are block-diagonal, once appropriately sorted).  If we repeat this for multiple such random splits we can extract different correlations.
The final choice is based on take a majority vote.

Like any good idea, this ideas has been had before.  A similar idea is available here:(https://stackoverflow.com/questions/31681373/making-svm-run-faster-in-python), which suggests
using a BaggingClassifier to automate the whole process.  
Of course, Random Forests are another option, with a similar goal.    

In [ ]:
from sklearn.svm import SVC
#just use bagging classifier on the whole list of SVMs
from sklearn.ensemble import BaggingClassifier

nfeature,nobs=X_train_counts.shape

In [ ]:
#Try to determine parameters gamma/C via cross-validation.
#Note that there is no need for explicit regularization?  Apparently in large dimensions, the parameters C/gamma (for penalty radius and width of basis function do a decent job in regularizing), since l1, l2 regularization don't work.  

In [ ]:
%pdb off
ind_sub,Xsub,label_sub=get_subset(0.01,X_train_counts,actual

Automatic pdb calling has been turned OFF


In [ ]:
#make the SVM model
svm=SVC(cache_size=750,gamma=0.01,C=10,class_weight='balanced')
#The bagging classifier of those
ensemble_svm=BaggingClassifier(svm,n_estimators=10,
bootstrap=False,n_jobs=3,max_samples=0.1,oob_score=False,verbose=True)

In [ ]:
frac_perc=0.2
#svm=SVC(cache_size=1000,verbose=True,gamma=0.1,C=0.5,class_weight='balanced')
indsub,Xsub,label_sub=get_subset(frac_perc,X_train_counts,df_train['toxic'].values)


In [ ]:
t0=time.time()
#use the ravel for reshaping?
ensemble_svm.fit(Xsub,label_sub.ravel())
svm_pred=ensemble_svm.predict(Xsub)
#test on a different subset of the training data
t1=time.time()
print('Time Elapsed:',t1-t0)
svm_stats=check_predictions(svm_pred,label_sub)

[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    3.8s finished


Time Elapsed: 62.2320671081543
(15288, 1) (15288, 1)
True Positive 0.03270538984824699. False Positive 0.0005232862375719519
False Negative 0.06397174254317112. True Negative 0.9027995813710099
Log-loss is 2.227579796059745
AUROC is 0.6688578514324013


[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:   20.2s finished


In [ ]:
frac_perc=0.04
#svm=SVC(cache_size=1000,verbose=True,gamma=0.1,C=0.5,class_weight='balanced')
indsub2,Xsub2,label_sub2=get_subset(frac_perc,X_train_counts,df_train['toxic'].values)
svm_pred2=ensemble_svm.predict(Xsub2)
svm_stats2=check_predictions(svm_pred2,label_sub2)

(3821, 1) (3821, 1)
True Positive 0.03245223763412719. False Positive 0.0010468463752944255
False Negative 0.06411934048678357. True Negative 0.9023815755037948
Log-loss is 2.25076119359395
AUROC is 0.66744230594102


[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    4.5s finished


In [ ]:
#Use Cross-validation to split, estimate score.
from sklearn.model_selection import GridSearchCV

gam_arr=np.logspace(-2,2,6)
C_arr=np.logspace(-2,2,6)
param_grid=dict(base_estimator__gamma=gam_arr,base_estimator__C=C_arr)

svm=SVC(cache_size=500,gamma=0.01,C=10,class_weight='balanced')
#The bagging classifier of those

ensemble_svm=BaggingClassifier(svm,n_estimators=10,
bootstrap=False,n_jobs=2,max_samples=0.1,oob_score=False,verbose=True)

#Uses stratified k-fold cross-validation.
gridsearch_svm=GridSearchCV(ensemble_svm,param_grid,error_score=0,scoring='neg_log_loss',cv=5)

#Then do grid search over that.  

In [ ]:
%pdb off
##So this search took around 5 hours on 2 cores, with 20% of data, 10 estimators.  
#gridsearch_svm.fit(Xsub,label_sub.ravel())

Automatic pdb calling has been turned OFF


In [ ]:
##Useful for getting parameters of bagged classifier.
#ensemble_svm.get_params()
gridsearch_svm.cv_results_

{'mean_fit_time': array([  8.515379  ,   9.01231294,   9.75185003,   9.73938398,   9.98681965,
         10.32953625,   9.1302866 ,   9.20638032,   9.61401176,   9.70504975,
         10.125845  ,  10.38082843,   8.5658195 ,   9.51361198,   9.58338413,
          9.42688198,   9.7426013 ,   9.23867021,   5.23752804,   8.05332527,
          8.26427064,   8.61298461,   8.92178097,   9.07002459,   4.95735879,
          7.89778571,   8.14328508,   8.49073753,   8.82482314,   8.98313236,
          5.27298045,   7.23655725,   8.26355944,   8.421489  ,   8.68758707,
          8.9933918 ]),
 'mean_score_time': array([ 16.90008068,  18.05262084,  19.87818789,  20.06860175,  20.97318258,
         21.40729494,  17.89313512,  18.53377829,  19.04816036,  19.9470829 ,
         21.13045225,  21.46793203,  16.92121625,  19.20889707,  18.34303741,
         19.25774665,  19.23158917,  17.89144721,   7.61561131,  13.21875992,
         16.03910851,  16.0835259 ,  17.37367401,  17.5949316 ,   6.19260755,
    

In [ ]:
scores=gridsearch_svm.cv_results_['mean_test_score'].reshape(len(gam_arr),len(C_arr))

In [ ]:
#gridsearch_svm.best_params_
#Results:{'base_estimator__C': 0.063095734448019331, 'base_estimator__gamma': 100.0}

{'base_estimator__C': 0.063095734448019331, 'base_estimator__gamma': 100.0}

In [ ]:
plt.figure()
plt.imshow(np.exp(scores))
plt.xticks(np.arange(len(gam_arr)),gam_arr,rotation=90)
plt.yticks(np.arange(len(C_arr)),C_arr)
plt.xlabel('gamma')
plt.ylabel('C')
plt.colorbar()
plt.show()

So this search took 5 hours or so (on 2 cores).  And suggests the two regions woth exploring are $C>1, \gamma\ll 1$, and $C<1,\gamma\gg 1$. 

## Randomized Fourier Features

The Tensorflow documentation includes a great idea for extending Kernel machines: use an sinusoidal mapping from the original space to another linear space.  The mapping depends on a Gaussian random variable, so when we take expectation values over the Gaussian variable, the result
of that expectation approximates the desired kernel.  Genius!
Ideas here:(https://www.tensorflow.org/tutorials/kernel_methods,
https://people.eecs.berkeley.edu/~brecht/papers/07.rah.rec.nips.pdf).
See also scikit-learn's Kernel Approximations methods, which implement the RBF kernel described below. 

LinearSVMs work quickly, but their full kernel counterparts are slow to train, scaling as $O(n_{sample}^3)$.
Instead, consider features like 
\begin{equation}
    z_{k}(\mathbf{x})=\cos(\mathbf{\omega}_{k}\cdot\mathbf{x}+b_{k}),
\end{equation}
where $\mathbf{x}\in \mathbb{R}^{d}, \omega\in \mathbb{R}^{d},\mathbf{b}_{k}\in\mathbb{R}$, and $\omega_{k}$, is a random Gaussian vector drawn from
\begin{equation}
    P(\omega) = (2\pi\sigma^2)^{-d/2} \exp\left(-\frac{\mathbf{\omega}^2}{2\sigma^2}\right),
\end{equation}
and $b_{k}$ is a uniform random variable drawn from $[0,2\pi)$.  Note that $z_{k}$ is a scalar.  But if we consider making $D$ draws of the random variables, then we can construct a vector $\mathbf{z}(\mathbf{x})=\sqrt{\frac{2}{D}}[z_{1},z_{2},\ldots, z_{D}]$,

The inner products on these new feature vectors for different input data are given y 
\begin{equation}
    \mathbf{z}(\mathbf{x})\cdot\mathbf{z}(\mathbf{y})=\frac{2}{D}\sum_{k=1}^{D} \cos(\mathbf{\omega}_{k}\cdot\mathbf{x}+b_{k})\cos(\mathbf{\omega}_{k}\cdot\mathbf{y}+b_{k}).
\end{equation}
This is essentially a Monte-Carlo estimate (with $D$ samples) of the probability distributions.  As $D\rightarrow \infty$, this converges to 
\begin{align}
    \mathbf{z}(\mathbf{x})\cdot\mathbf{z}(\mathbf{y})&\approx \int d\mathbf{\omega}\int db\,P(\omega)p(b)
    2\cos(\mathbf{\omega}\cdot\mathbf{x}+b)\cos(\mathbf{\omega}\cdot\mathbf{y}+b)\\
&=\frac{1}{2\pi}\frac{1}{(2\pi \sigma^2)^{D/2}}\int d\mathbf{\omega}\int_0^{2\pi} db\,e^{-(\mathbf{\omega})^2/(2\sigma^2)}
    2\cos(\mathbf{\omega}\cdot\mathbf{x}+b)\cos(\mathbf{\omega}\cdot\mathbf{y}+b) \\
&=\frac{1}{2\pi}\frac{1}{(2\pi \sigma^2)^{D/2}}\int d\mathbf{\omega}\int_0^{2\pi} db\,e^{-(\mathbf{\omega})^2/(2\sigma^2)}
    \bigg(\cos[\mathbf{\omega}\cdot(\mathbf{x}+\mathbf{y})+b]+\cos[\mathbf{\omega}\cdot(\mathbf{x}-\mathbf{y})]\bigg),
\end{align}
where we used a double-angle formula on the cosines.  The Gaussian and uniform integrals can be carried out, with the result
\begin{align}
    \mathbf{z}(\mathbf{x})\cdot\mathbf{z}(\mathbf{y})&\approx 
&=\,e^{-(\mathbf{x-y})^2/(2\sigma^2)}.
\end{align}
The same idea can be extended for any $P(\mathbf{\omega})$ to get the desired kernel, provided it has a nice Fourier transform.

One thing noted in the docs is that this works well for smooth data, but can require a lot of components if there is a significant random component, such as trying to detect fractal structures like forests in images.  


In [ ]:
#Let's try to compare that with a full SVM on the same data.
t0=time.time()
full_svm=SVC(cache_size=1000,verbose=True,gamma=0.01,C=10,class_weight='balanced')
full_svm.fit(Xsub,label_sub.ravel())
full_svm_pred=full_svm.predict(Xsub)
t1=time.time()
print('Training time',t1-t0)
svm_stats3=check_predictions(full_svm_pred,label_sub)

[LibSVM]

Training time 78.60097455978394
(15288, 1) (15288, 1)
True Positive 0.09641548927263213. False Positive 0.004120879120879121
False Negative 0.00026164311878597594. True Negative 0.8992019884877027
Log-loss is 0.15137025072587212
AUROC is 0.9963658641979543
